In [1]:
### Dependencies and Setup
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy import stats
from scipy.stats import linregress
import gmaps

import json
import pprint as pprint

#import api key
from api_keys import gmaps_api_key

In [2]:
location_data = pd.read_csv('../WeatherPy/output_data/cities.csv')
location_data.head()

,city,country,id,lat,lng,temp,humidity,clouds,wind_speed
0,Hithadhoo,MV,1282256,-0.60,73.08,83.28,78,79,14.74
1,Avarua,CK,4035715,-21.21,-159.78,84.20,83,80,9.17
2,Klaksvik,FO,2618795,62.23,-6.59,44.60,100,100,23.04
3,Hilo,US,5855927,19.73,-155.09,72.27,69,90,9.17
4,Bandarbeyla,SO,64814,9.49,50.81,79.20,79,100,19.95


In [3]:
gmaps.configure(api_key=gmaps_api_key)

In [4]:
#create heat map for humidity
#store lat and lng into locations
locations = location_data[['lat', 'lng']].astype(float)
#convert humidity to float and store
humidity = location_data['humidity'].astype(float)

In [8]:
#create humidity heatmap layer
fig = gmaps.figure(zoom_level=2, center=(0, 0))

humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
#locate rows in the ideal temperature range
ideal_spot = location_data.loc[(location_data['temp'] >= 70) & (location_data['temp'] <= 80)]
ideal_spot_2 = ideal_spot.loc[ideal_spot['wind_speed'] <= 15]
ideal_spot_3 = ideal_spot_2.loc[ideal_spot_2['clouds'] <= 35]
ideal_vacation = ideal_spot_3.loc[(ideal_spot_3['humidity'] >= 35) & (ideal_spot_3['humidity'] <= 65)]
ideal_vacation.to_csv('output_data/ideal_vacation.csv', index=False)
ideal_vacation

,city,country,id,lat,lng,temp,humidity,clouds,wind_speed
38,Puerto Ayora,EC,3652764,-0.74,-90.35,77.00,57,23,10.31
171,Rocha,UY,3440777,-34.48,-54.33,70.83,49,0,7.90
172,Urubicha,BO,3901969,-15.38,-62.95,77.00,55,31,1.50
215,Alibag,IN,1279027,18.64,72.88,75.20,61,20,4.70
242,Port Lincoln,AU,2063036,-34.73,135.87,75.43,48,0,3.85
366,Kahului,US,5847411,20.89,-156.47,79.25,61,20,11.41
501,Moron,AR,3430545,-34.65,-58.62,78.93,54,0,3.36
525,Itaqui,BR,3460629,-29.13,-56.55,70.07,53,0,3.38


In [ ]:
#dictionary to update each iteration
params = {
    'radius': 5000,
    'types': 'hotel',
    'keyword': 'hotel',
    'key': gmaps_api_key
}

#identify hotel using lat/lng
for index, row in ideal_vacation.iterrows():
    lat = row['lat']
    lng = row['lng']
    
    #change location while leaving original parameters in place
    params['location'] = f'{lat},{lng}'
    
    #
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    #make request and print URL
    name_address 
    

